In [1]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx

In [7]:
main_arr = []
with open('D:/Datasets/MIMOS/traffic_flow_26082020.json') as f:
    here_traffic_flow = f.readlines()
    for line in here_traffic_flow:
        
        
        line = json.loads(line)
        line = line['_source']
        #print(line)
        for flow_item_s in line['FIS']:
            for flow_item in flow_item_s['FI']:
                #print(flow_item)
                temp_arr = []
                for tmc_item in flow_item['TMC']:
                    #print(flow_item['TMC'][tmc_item])
                    temp_arr.append(flow_item['TMC'][tmc_item])
                
                ## To get SHP
                #shp_values = flow_item['SHP'][0]['value'][0]
                
                arr_lon = []
                arr_lat = []
                
                fc_temp = flow_item['SHP'][0]['FC']
                
                for shp in flow_item['SHP']:
                    shp_values = shp['value'][0]
                    coordinates_arr = shp_values.split(' ')
                    #print(coordinates_arr)
                    
                    for item in coordinates_arr:
                        if len(item) > 0:
                            item = item.split(',')
                            arr_lat.append(float(item[0]))
                            arr_lon.append(float(item[1]))
                            
                    #if fc_temp != shp['FC']:
                        #print(fc_temp, shp['FC'])
                        #print(line['PBT'])
                        #print(line['DE'])
                        #print('FC Diff')
                    
                polygon_geom = LineString(zip(arr_lon, arr_lat))
                #temp_arr.append(flow_item['SHP'][0]['value'])
                temp_arr.append(polygon_geom)
                
                temp_arr.append(flow_item['SHP'][0]['FC'])
                
                
                ## To get all SHP
                """
                for cf_item in flow_item['SHP'][0]:   
                    temp_arr.append(flow_item['SHP'][0][cf_item])
                
                
                for cf_item in flow_item['CF'][0]:   
                    temp_arr.append(flow_item['CF'][0][cf_item])
                """
                try:
                    temp_arr.append(flow_item['CF'][0]['SP'])
                    temp_arr.append(flow_item['CF'][0]['SU'])
                    temp_arr.append(flow_item['CF'][0]['FF'])
                    temp_arr.append(flow_item['CF'][0]['JF'])
                    temp_arr.append(flow_item['CF'][0]['CN'])
                
                    temp_arr.append(line['PBT'])
                    temp_arr.append(line['DE'])
                
                
                
                    #print(len(temp_arr))
                    if len(temp_arr) == 13:
                        main_arr.append(temp_arr)
                except:
                    pass

In [8]:
here_traffic_flow_df = pd.DataFrame(main_arr)

In [9]:
here_traffic_flow_df.columns = ['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                            'Geometry', 'FC',
                             'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT', 'DE']



In [10]:
here_traffic_flow_df.tail()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,Geometry,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE
353395,16896,E1/Lebuhraya Utara-Selatan,+,0.84026,"LINESTRING (100.51678 5.65491, 100.51649 5.654...",4,40.0,40.0,46.1,0.89660,0.7,2020-08-26T07:19:35Z,3052
353396,16895,Jalan Peruda,+,0.43107,"LINESTRING (100.50961 5.65252, 100.50944 5.652...",4,51.0,51.0,51.9,0.23121,0.7,2020-08-26T07:19:35Z,3052
353397,16894,1/Jalan Badlishah,+,0.64690,"LINESTRING (100.50587 5.6515, 100.50531 5.6515...",4,44.0,44.0,53.9,1.70686,0.7,2020-08-26T07:19:35Z,3052
353398,16901,225/Jalan Lencongan Timur,+,0.03909,"LINESTRING (100.5285 5.66732, 100.52848 5.6671...",4,30.0,30.0,24.0,0.00000,0.7,2020-08-26T07:19:35Z,Jalan Kelab Cinta Sayang
353399,16900,3052,+,1.98643,"LINESTRING (100.52845 5.66697, 100.5284 5.6663...",4,50.0,50.0,37.4,0.00000,0.7,2020-08-26T07:19:35Z,Jalan Kelab Cinta Sayang


In [11]:
print(len(here_traffic_flow_df))

353400


In [12]:
here_traffic_flow_gdf = gpd.GeoDataFrame(here_traffic_flow_df, geometry='Geometry')

In [14]:
pwd = 'C:\\Users\\tanji\\Desktop\\MIMOS_Geospatial'
try:
    os.mkdir('traffic_flow_shp/traffic_flow_26082020')
except:
    pass
#os.mkdir(pwd + '/traffic_flow/traffic_flow_26082020/')
here_traffic_flow_gdf.to_file('traffic_flow_shp/traffic_flow_26082020/traffic_flow_26082020_0700_0800.shp')

In [ ]:
pwd

In [7]:
here_traffic_flow_df.to_csv('traffic_flow.csv', index=False)

In [ ]:
['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                            'Geometry', 'FC',
                               'CF_TY', 'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT', 'DE', 'mid', 'LI', 'timestamp']

In [31]:
#test = here_traffic_flow_df.drop(['Geometry'], axis=1)

test = here_traffic_flow_df[['TMC_DE', 'TMC_QD', 'TMC_LE', 'FC',
                                'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT'
                               ]]

In [46]:
here_traffic_flow_df_duplicated = test[test.duplicated()]
print(len(here_traffic_flow_df_duplicated))
here_traffic_flow_df_duplicated.head()

24


,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT
24919,Lebuh Jenahak/Jalan Tenggiri,+,1.07911,1,72,72.0,90.0,2.93333,0.7,2020-08-26T07:29:35Z
30846,Lebuh Jenahak/Jalan Tenggiri,+,1.07911,1,72,72.0,90.0,2.93333,0.7,2020-08-26T07:44:35Z
43048,Lebuh Jenahak/Jalan Tenggiri,+,1.07911,1,72,72.0,90.0,2.93333,0.7,2020-08-26T07:49:35Z
48299,Persiaran Tasik,-,0.04505,3,20,20.0,20.0,0.00000,0.7,2020-08-26T07:49:35Z
58270,Lebuh Jenahak/Jalan Tenggiri,+,1.07911,1,72,72.0,90.0,2.93333,0.7,2020-08-26T07:59:35Z


In [42]:
lol = here_traffic_flow_df.iloc[30846,:]

In [43]:
lol2 = here_traffic_flow_df.iloc[43048,:]

In [44]:
lol

TMC_PC                                                    2640
TMC_DE                            Lebuh Jenahak/Jalan Tenggiri
TMC_QD                                                       +
TMC_LE                                                 1.07911
Geometry     LINESTRING (100.40616 5.39502, 100.40605 5.394...
FC                                                           1
CF_TY                                                       TR
CF_SP                                                       72
CF_SU                                                       72
CF_FF                                                       90
CF_JF                                                  2.93333
CF_CN                                                      0.7
PBT                                       2020-08-26T07:44:35Z
DE                                     Lebuhraya Utara-Selatan
mid                       151199eb-d80f-45d8-87b8-e8809d692052
LI                                                   F0

In [45]:
lol2

TMC_PC                                                    2640
TMC_DE                            Lebuh Jenahak/Jalan Tenggiri
TMC_QD                                                       +
TMC_LE                                                 1.07911
Geometry     LINESTRING (100.40616 5.39502, 100.40605 5.394...
FC                                                           1
CF_TY                                                       TR
CF_SP                                                       72
CF_SU                                                       72
CF_FF                                                       90
CF_JF                                                  2.93333
CF_CN                                                      0.7
PBT                                       2020-08-26T07:49:35Z
DE                                     Lebuhraya Utara-Selatan
mid                       151199eb-d80f-45d8-87b8-e8809d692052
LI                                                   F0